# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 3:14PM,244305,10,MSP213161,"Methapharm, Inc.",Executing
1,Aug 16 2022 3:14PM,244305,10,MSP213169,"Methapharm, Inc.",Executing
2,Aug 16 2022 2:57PM,244303,10,0085939281,ISDIN Corporation,Released
3,Aug 16 2022 2:57PM,244303,10,0085939282,ISDIN Corporation,Released
4,Aug 16 2022 2:57PM,244303,10,0085939285,ISDIN Corporation,Released
5,Aug 16 2022 2:57PM,244303,10,0085939286,ISDIN Corporation,Released
6,Aug 16 2022 2:57PM,244303,10,0085939284,ISDIN Corporation,Released
7,Aug 16 2022 2:57PM,244303,10,0085939283,ISDIN Corporation,Released
8,Aug 16 2022 2:49PM,244302,16,8380674,Sartorius Bioprocess Solutions,Released
9,Aug 16 2022 2:33PM,244300,10,0085939238,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,244299,Released,10
30,244300,Released,8
31,244302,Released,1
32,244303,Released,6
33,244305,Executing,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244299,NaN,9.0,10.0
244300,NaN,NaN,8.0
244302,NaN,NaN,1.0
244303,NaN,NaN,6.0
244305,NaN,2.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244215,53.0,11.0,10.0
244216,19.0,9.0,10.0
244222,6.0,3.0,3.0
244227,2.0,0.0,6.0
244245,3.0,5.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244215,53,11,10
244216,19,9,10
244222,6,3,3
244227,2,0,6
244245,3,5,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244215,53,11,10
1,244216,19,9,10
2,244222,6,3,3
3,244227,2,0,6
4,244245,3,5,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244215,53,11,10
1,244216,19,9,10
2,244222,6,3,3
3,244227,2,,6
4,244245,3,5,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc."
2,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation
8,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions
9,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation
17,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd"
36,Aug 16 2022 1:59PM,244298,12,Hush Hush
38,Aug 16 2022 1:38PM,244290,10,Beach Products Inc
39,Aug 16 2022 1:32PM,244289,10,Beach Patient Assistance
43,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC"
50,Aug 16 2022 12:24PM,244275,16,Seqirus


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",,2,
1,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,,,6
2,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,,,1
3,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,,,8
4,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",,9,10
5,Aug 16 2022 1:59PM,244298,12,Hush Hush,,1,1
6,Aug 16 2022 1:38PM,244290,10,Beach Products Inc,,1,
7,Aug 16 2022 1:32PM,244289,10,Beach Patient Assistance,,,4
8,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",,,7
9,Aug 16 2022 12:24PM,244275,16,Seqirus,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",,2,
1,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6,,
2,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1,,
3,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8,,
4,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",10,9,
5,Aug 16 2022 1:59PM,244298,12,Hush Hush,1,1,
6,Aug 16 2022 1:38PM,244290,10,Beach Products Inc,,1,
7,Aug 16 2022 1:32PM,244289,10,Beach Patient Assistance,4,,
8,Aug 16 2022 1:04PM,244285,10,"Snap Medical Industries, LLC",7,,
9,Aug 16 2022 12:24PM,244275,16,Seqirus,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",,2,
1,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6,,
2,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1,,
3,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8,,
4,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",10,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",NaN,2.0,NaN
1,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6.0,NaN,NaN
2,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8.0,NaN,NaN
4,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",10.0,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",0.0,2.0,0.0
1,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6.0,0.0,0.0
2,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8.0,0.0,0.0
4,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",10.0,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1709999,31.0,3.0,2.0
12,244298,1.0,1.0,0.0
15,732653,23.0,23.0,78.0
16,977112,4.0,0.0,0.0
19,244299,10.0,9.0,0.0
20,977028,50.0,6.0,3.0
22,732772,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1709999,31.0,3.0,2.0
1,12,244298,1.0,1.0,0.0
2,15,732653,23.0,23.0,78.0
3,16,977112,4.0,0.0,0.0
4,19,244299,10.0,9.0,0.0
5,20,977028,50.0,6.0,3.0
6,22,732772,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,3.0,2.0
1,12,1.0,1.0,0.0
2,15,23.0,23.0,78.0
3,16,4.0,0.0,0.0
4,19,10.0,9.0,0.0
5,20,50.0,6.0,3.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,12,Released,1.0
2,15,Released,23.0
3,16,Released,4.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,2.0,0.0,78.0,0.0,0.0,3.0,0.0
Executing,3.0,1.0,23.0,0.0,9.0,6.0,0.0
Released,31.0,1.0,23.0,4.0,10.0,50.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,2.0,0.0,78.0,0.0,0.0,3.0,0.0
1,Executing,3.0,1.0,23.0,0.0,9.0,6.0,0.0
2,Released,31.0,1.0,23.0,4.0,10.0,50.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,2.0,0.0,78.0,0.0,0.0,3.0,0.0
1,Executing,3.0,1.0,23.0,0.0,9.0,6.0,0.0
2,Released,31.0,1.0,23.0,4.0,10.0,50.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()